This is a Notebook created with Insturctions for Import data feature. 

It is important to follow the Pre-steps listed below carefully to ensure the proper working, else you might face issues and might have to do backtracking.
You can give an appropriate names to your resources by updating the name in the corresponding YAML files in the samples or by creating your own YAML using the sample and referencing it in the following when you are ready to create. These are just samples provided for you to get familiar with the required parameters that needs to be passed to create a new resource.


**1. Create test workspace with a compute instance 'cpu-instance'**

If you don't have an Azure subscription, sign up to try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/) today.
and follow the instructions to create a [AzureML workspace and compute instance](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources)

**2. Install SDKv2 private preview release**

In [ ]:
pip install azure-ai-ml==0.0.71493720 --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-v2-cli-e2e-test/71493720/

In [9]:
from azure.identity import ManagedIdentityCredential
from azure.ai.ml import MLClient

subscription_id = '<your subscription here>'
resource_group = '<your rg here>'
workspace = '<your ws here>'
ml_client = MLClient(ManagedIdentityCredential(), subscription_id, resource_group, workspace)

**Check for existing Connections**

In [ ]:
#ml_client.connections.get(name=<connection_name>)

connection_list = ml_client.connections.list()

for conn in connection_list:
    print(conn)

**OR create a new connection by the follwing instructions in [create_ws_connections.ipynb](./create_ws_connection_sdk_v2.ipynb)**

**Import Job With Az SQL as an example using DatabaseImportSource class option**

*NOTE:* The same can be used for other types such as Snowflake, Synapse and Az SQL jst by pointing to the right type, connection and query
The `path` here is an example path and `{name}` ensures that the path is unique by substituting it with the name of the `job`

In [ ]:
from azure.ai.ml.entities import FileImportSource,DatabaseImportSource, ImportJob
from azure.ai.ml import import_job
from azure.ai.ml import Output
from azure.ai.ml import load_job

data_output = Output(type='mltable', path='azureml://datastores/workspaceblobstore/paths/azureml/{name}/output_dir/azsql')
data_source = DatabaseImportSource(type='azuresqldb', connection='azureml:<your_connection_name>', query='<your query here>')

data_import_job = ImportJob(source=data_source, output=data_output)
returned_job = ml_client.jobs.create_or_update(data_import_job)


**Import Job With Snowflake as an example using DatabaseImportSource class option**

*NOTE:* The same can be used for other types such as Snowflake, Synapse and Az SQL jst by pointing to the right type, connection and query
The `path` here is an example path and `{name}` ensures that the path is unique by substituting it with the name of the `job`

In [ ]:
from azure.ai.ml.entities import FileImportSource,DatabaseImportSource, ImportJob
from azure.ai.ml import import_job
from azure.ai.ml import Output
from azure.ai.ml import load_job

data_output = Output(type='mltable', path='azureml://datastores/workspaceblobstore/paths/azureml/{name}/output_dir/snowflake')
data_source = DatabaseImportSource(type='snowflake', connection='azureml:my_snowflake_connection', query='select * from TPCH_SF1000.PARTSUPP')

data_import_job = ImportJob(source=data_source, output=data_output)
returned_job = ml_client.jobs.create_or_update(data_import_job)


**Import Job With S3 as an example using FileImportSource class option**

*NOTE:* The `path` here is an example path and `{name}` ensures that the path is unique by substituting it with the name of the `job`

In [ ]:
from azure.ai.ml.entities import FileImportSource,DatabaseImportSource, ImportJob
from azure.ai.ml import import_job
from azure.ai.ml import Output
from azure.ai.ml import load_job

data_output = Output(type='uri_folder', path='azureml://datastores/workspaceblobstore/paths/azureml/{name}/output_dir/')
data_source = FileImportSource(type='s3', connection='azureml:my_s3_connection', path='test1/*')

data_import_job = ImportJob(source=data_source, output=data_output)
returned_job = ml_client.jobs.create_or_update(data_import_job)

**Import Job With Snowflake as an example with loading the YAML option**

*NOTE:* The same can be used for other types such as S3. Az Synapse and Az SQL jst by pointing to the right YAML

In [ ]:
data_import_job = load_job(path="./job/import_job_test_snowflake.yml")
returned_job: ImportJob = ml_client.jobs.create_or_update(job=data_import_job)